In [2]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [3]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

# 导入数据

In [4]:
# download training data from open datasets
training_data = datasets.FashionMNIST(
    root='data',
    train=True,
    download=True,
    transform=ToTensor()
)

# download test data from open datasets
test_data = datasets.FashionMNIST(
    root='data',
    train=False,
    download=True,
    transform=ToTensor()
)

# 查看数据内容

In [5]:
training_data.data.size()
test_data.data.size()

torch.Size([60000, 28, 28])

torch.Size([10000, 28, 28])

训练数据包含6万张图片，28 * 28大小。
测试数据包含1万张图片，28 * 28大小。

In [13]:
batch_size = 64

# Create dataloader
train_dataloader =DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"shape of X [N, C, H, W]: {X.shape}")
    print(f"shape of y: {y.shape} {y.dtype}")
    break

shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
shape of y: torch.Size([64]) torch.int64


# 定义模型

In [10]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [6]:
# Define a model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [7]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


# 定义optimizer & loss function

In [8]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

# 训练过程定义

In [15]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # backpropgation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f} [{current}:>5d]/{size:>5d}")


# 测试过程定义

In [12]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, AvgLoss: {test_loss:>8f} \n")

# 训练模型并展示训练过程

In [16]:
epoch = 5
for t in range(epoch):
    print(f'Epoch {t+1}\n ------------------------------')
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("DONE")

Epoch 1
 ------------------------------
loss: 2.306754 [0:>5d]/60000
loss: 2.289436 [6400:>5d]/60000
loss: 2.268823 [12800:>5d]/60000
loss: 2.267000 [19200:>5d]/60000
loss: 2.228488 [25600:>5d]/60000
loss: 2.208755 [32000:>5d]/60000
loss: 2.215133 [38400:>5d]/60000
loss: 2.180605 [44800:>5d]/60000
loss: 2.178737 [51200:>5d]/60000
loss: 2.143647 [57600:>5d]/60000
Test Error: 
 Accuracy: 42.7%, AvgLoss: 2.132832 

Epoch 2
 ------------------------------
loss: 2.146586 [0:>5d]/60000
loss: 2.132410 [6400:>5d]/60000
loss: 2.071460 [12800:>5d]/60000
loss: 2.100368 [19200:>5d]/60000
loss: 2.011267 [25600:>5d]/60000
loss: 1.966015 [32000:>5d]/60000
loss: 1.992899 [38400:>5d]/60000
loss: 1.907786 [44800:>5d]/60000
loss: 1.919832 [51200:>5d]/60000
loss: 1.848010 [57600:>5d]/60000
Test Error: 
 Accuracy: 50.4%, AvgLoss: 1.836061 

Epoch 3
 ------------------------------
loss: 1.872179 [0:>5d]/60000
loss: 1.836189 [6400:>5d]/60000
loss: 1.716229 [12800:>5d]/60000
loss: 1.775487 [19200:>5d]/60000
l

# 储存模型

In [18]:
torch.save(model.state_dict(), 'model.pth')
print('Save PyTorch Model State to model.pth')

Save PyTorch Model State to model.pth


# 加载模型

In [ ]:
model = NeuralNetwork()
model.load_state_dict(torch.load('model.pth'))

In [ ]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')